In [1]:
import os
import json
import requests

In [2]:
%load_ext dotenv
%dotenv

In [3]:
id_catalog = os.environ["ID_CATALOG_YANDEX"]
api_key = os.environ["API_KEY_YANDEX"]

In [7]:
def pred_history(message):
    prompt = {
        "modelUri": f"gpt://{id_catalog}/yandexgpt/latest",  # YandexGPT 3 Pro
        "completionOptions": {"stream": False, "temperature": 0.6, "maxTokens": "2000"},
        "messages": [
            {
                "role": "system",
                "text": "Ты русский писатель детских рассказов. Всегда возвращаешь текст только на русском.",
            },
            {
                "role": "user",
                "text": message,
            },
        ],
    }

    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Api-Key {api_key}",
    }

    response = requests.post(url, headers=headers, json=prompt)

    result = json.loads(response.text)

    return result["result"]["alternatives"][0]["message"]["text"]

In [8]:
message = (
    "Write a small 50-word story starring a girl Katya in the park with a teddy bear"
)

In [9]:
result = pred_history(message)

result

'В парке на скамейке сидела девочка Катя и читала книгу. Рядом с ней сидел её верный друг – плюшевый медведь. Вдруг подул сильный ветер, и книга чуть не улетела. Но Катя успела поймать её, а мишка удержал шляпку девочки. Катя улыбнулась своему другу и обняла его.'